In [1]:
import pandas as pd

damages_df = pd.read_csv('scrape_parse/Damages.csv', index_col=0)
bomb_events_df = pd.read_csv('scrape_parse/BombEvents.csv', index_col=0)
rounds_df = pd.read_csv('scrape_parse/Rounds.csv', index_col=0)
kills_df = pd.read_csv('scrape_parse/Kills.csv', index_col=0)


In [2]:
rounds_df.head()

,MatchId,MapName,RoundNum,StartTick,FreezeTimeEnd,EndTick,EndOfficialTick,TScore,CTScore,WinningSide,WinningTeam,LosingTeam,RoundEndReason,CTStartEqVal,CTBuyType,TStartEqVal,TBuyType
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,13232,15794,27867,27867,0,0,CT,SWS Gaming,Santos e-Sports,BombDefused,4350,Pistol,4550,Pistol
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,2,28507,31070,46939,46945,0,1,T,Santos e-Sports,SWS Gaming,TargetBombed,20800,Half Buy,16150,Half Buy
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,3,47585,50138,64531,64534,1,1,T,Santos e-Sports,SWS Gaming,TargetBombed,16350,Half Buy,23800,Full Buy
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,4,65174,67724,81080,81090,2,1,T,Santos e-Sports,SWS Gaming,TargetBombed,6200,Half Buy,25300,Full Buy
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,5,81730,84283,98592,98601,3,1,T,Santos e-Sports,SWS Gaming,TerroristsWin,24150,Full Buy,25400,Full Buy


In [3]:
game_state_cols = ['MatchId', 'MapName', 'RoundNum', 'TicksSinceStart', 'CTStartEqVal', 'TStartEqVal', 'CTAlive', 'TAlive', 'CTTotalHP', 'TTotalHP', 'BombPlantedA', 'BombPlantedB', 'CTClosestA', 'TClosestA', 'CTClosestB', 'TClosestB']

## Remove events before and after the round

In [4]:
# TODO: Check that only the post-round time is getting cut (weird since the endtick and officialendtick are practically the same)

In [5]:
def keep_in_round_time(df):
    # df needs to have "MatchId", "MapName", "RoundNum" columns
    merged_df = df.merge(rounds_df[['MatchId', 'MapName', 'RoundNum', 'FreezeTimeEnd', 'EndTick']], how='left', on=['MatchId', 'MapName', 'RoundNum'], validate='many_to_one')
    merged_dropped_df = merged_df[(merged_df.Tick > merged_df.FreezeTimeEnd) & (merged_df.Tick <= merged_df.EndTick)]
    merged_dropped_df.insert(3, 'RoundTick', merged_dropped_df.Tick - merged_dropped_df.FreezeTimeEnd)
    return merged_dropped_df.drop(['Tick', 'FreezeTimeEnd', 'EndTick'], axis=1)

In [6]:
# m_bomb_events_df = bomb_events_df.merge(rounds_df[['MatchId', 'MapName', 'RoundNum', 'FreezeTimeEnd', 'EndTick']], how='left', on=['MatchId', 'MapName', 'RoundNum'], validate='many_to_one')
# m_bomb_events_df[(m_bomb_events_df.Tick <= m_bomb_events_df.FreezeTimeEnd) | (m_bomb_events_df.Tick > m_bomb_events_df.EndTick)]

In [7]:
damages_df = keep_in_round_time(damages_df)
bomb_events_df = keep_in_round_time(bomb_events_df)
kills_df = keep_in_round_time(kills_df)

## Bomb Plants

In [8]:
bomb_events_df.head()

,MatchId,MapName,RoundNum,RoundTick,Second,PlayerSteamId,PlayerName,PlayerTeam,PlayerX,PlayerY,PlayerZ,BombAction,BombSite
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,7625,59.570312,76561198169934019,★ ⑳ MaLLby,Santos e-Sports,-2549.361572,645.968750,480.03125,plant,A
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,12073,94.320312,76561197963277583,★ ㉑ matios,SWS Gaming,-2507.430908,645.319946,480.03125,defuse,A
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,2,10616,82.937500,76561198148561509,★ ㉑ cass1n,Santos e-Sports,-954.525574,-31.595411,96.03125,plant,B
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,3,9136,71.375000,76561198169934019,★ ⑳ MaLLby,Santos e-Sports,-1166.488892,-79.727104,98.03125,plant,B
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,4,8108,63.343750,76561198128804839,★ ㉑ voltera,Santos e-Sports,-1166.488892,-73.471153,98.03125,plant,B


In [9]:
bomb_events_df['BombPlantedA'] = (bomb_events_df['BombSite'] == 'A') & (bomb_events_df['BombAction'] == 'plant')
bomb_events_df['BombPlantedB'] = (bomb_events_df['BombSite'] == 'B') & (bomb_events_df['BombAction'] == 'plant')
bomb_events_df['AttackerSide'] = bomb_events_df['BombAction'].map({'plant': 'T', 'defuse': 'CT'})
bomb_events_df['VictimSide'] = bomb_events_df['BombAction'].map({'plant': 'CT', 'defuse': 'T'})
bomb_events_df['AttackerSteamId'] = bomb_events_df['PlayerSteamId']

bomb_state_df = bomb_events_df.drop(columns=['Second', 'PlayerX', 'PlayerY', 'PlayerZ', 'BombSite', 'PlayerName', 'PlayerTeam', 'BombAction', 'PlayerSteamId'])
bomb_state_df.set_index(['MatchId', 'MapName', 'RoundNum', 'RoundTick'], inplace=True)
bomb_state_df.head()

BombPlantedA  \
MatchId                                            MapName     RoundNum RoundTick                 
Liga-Gamers-Club-2021-Serie-A-April-Cup-santos-... de_overpass 1        7625               True   
                                                                        12073             False   
                                                               2        10616             False   
                                                               3        9136              False   
                                                               4        8108              False   

                                                                                   BombPlantedB  \
MatchId                                            MapName     RoundNum RoundTick                 
Liga-Gamers-Club-2021-Serie-A-April-Cup-santos-... de_overpass 1        7625              False   
                                                                        12073             False   
                                                               2        10616              True   
                                                               3        9136               True   
                                                               4        8108               True   

                                                                                  AttackerSide  \
MatchId                                            MapName     RoundNum RoundTick                
Liga-Gamers-Club-2021-Serie-A-April-Cup-santos-... de_overpass 1        7625                 T   
                                                                        12073               CT   
                                                               2        10616                T   
                                                               3        9136                 T   
                                                               4        8108                 T   

                                                                                  VictimSide  \
MatchId                                            MapName     RoundNum RoundTick              
Liga-Gamers-Club-2021-Serie-A-April-Cup-santos-... de_overpass 1        7625              CT   
                                                                        12073              T   
                                                               2        10616             CT   
                                                               3        9136              CT   
                                                               4        8108              CT   

                                                                                     AttackerSteamId  
MatchId                                            MapName     RoundNum RoundTick                     
Liga-Gamers-Club-2021-Serie-A-April-Cup-santos-... de_overpass 1        7625       76561198169934019  
                                                                        12073      76561197963277583  
                                                               2        10616      76561198148561509  
                                                               3        9136       76561198169934019  
                                                               4        8108       76561198128804839

## Kills and Damage

In [10]:
# Drop C4 kills and damages since those happen when the round ends
kills_df = kills_df[kills_df.Weapon != 'C4']
damages_df = damages_df[damages_df.Weapon != 'C4']

In [11]:
kills_df = kills_df[['MatchId', 'MapName', 'RoundNum', 'RoundTick', 'AttackerSteamId', 'AttackerSide', 'VictimSteamId', 'VictimSide']]
kills_df['AttackerSteamId'] = kills_df['AttackerSteamId'].astype(str)
kills_df['VictimSteamId'] = kills_df['VictimSteamId'].astype(str)
kills_df.head()

,MatchId,MapName,RoundNum,RoundTick,AttackerSteamId,AttackerSide,VictimSteamId,VictimSide
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5173,7.656119814856152e+16,T,76561198286138224,CT
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5321,7.656119814856152e+16,T,76561198135228658,CT
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5465,7.656119796327758e+16,CT,76561198148561509,T
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,6005,7.656119816993403e+16,T,76561198061789000,CT
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,6997,7.656119816993403e+16,T,76561198051070311,CT


In [12]:
kills_df.VictimSteamId = kills_df.VictimSteamId.astype(float).astype(str)

In [13]:
damages_df = damages_df[['MatchId', 'MapName', 'RoundNum', 'RoundTick', 'HpDamageTaken', 'AttackerSteamId', 'AttackerSide', 'VictimSteamId', 'VictimSide']]
damages_df['AttackerSteamId'] = damages_df['AttackerSteamId'].astype(str)
damages_df['VictimSteamId'] = damages_df['VictimSteamId'].astype(str)
damages_df.head()

,MatchId,MapName,RoundNum,RoundTick,HpDamageTaken,AttackerSteamId,AttackerSide,VictimSteamId,VictimSide
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5149,21,7.65611981485615e+16,T,76561198286138224,CT
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5173,79,7.65611981485615e+16,T,76561198286138224,CT
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5321,100,7.65611981485615e+16,T,76561198135228658,CT
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5465,100,7.656119796327758e+16,CT,76561198148561509,T
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5965,11,7.656119816993402e+16,T,76561198061789000,CT


In [14]:
# Combine simultaneous instances of damage from the same player to the same player
kills_damage_key = ['MatchId', 'MapName', 'RoundNum', 'RoundTick', 'AttackerSteamId', 'AttackerSide', 'VictimSteamId', 'VictimSide']
damages_df = damages_df.groupby(kills_damage_key, as_index=False).sum()
damages_df['HpDamageTaken'] = damages_df.HpDamageTaken.apply(lambda x: min([x, 100]))  # Bugged/laggy damages instances > 100 (shouldn't be possible)


In [15]:
# Check that all kills have a corresponding damage event
inner_merge_df = kills_df.merge(damages_df, on=kills_damage_key)

# if len(inner_merge_df) != len(kills_df):
#     raise ValueError(f'{len(inner_merge_df)} kills have corresponding damage event, {len(kills_df)} kills in total')

In [16]:
kills_df.head()

,MatchId,MapName,RoundNum,RoundTick,AttackerSteamId,AttackerSide,VictimSteamId,VictimSide
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5173,7.656119814856152e+16,T,7.656119828613822e+16,CT
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5321,7.656119814856152e+16,T,7.656119813522866e+16,CT
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,5465,7.656119796327758e+16,CT,7.65611981485615e+16,T
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,6005,7.656119816993403e+16,T,7.6561198061789e+16,CT
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,6997,7.656119816993403e+16,T,7.65611980510703e+16,CT


### Merging kills into damage

In [17]:
kills_df['IsKill'] = True
damages_df = pd.merge(damages_df, kills_df, how='left', on=kills_damage_key, validate='one_to_one')
damages_df['IsKill'].fillna(False, inplace=True)
damages_df.head()

,MatchId,MapName,RoundNum,RoundTick,AttackerSteamId,AttackerSide,VictimSteamId,VictimSide,HpDamageTaken,IsKill
0,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,646,7.656119811125186e+16,T,76561198422389693,CT,85,False
1,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,928,7.656119819470005e+16,CT,76561198111251858,T,11,False
2,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,1038,7.656119811125186e+16,T,76561198194700054,CT,89,False
3,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,1110,7.656119811125186e+16,T,76561198194700054,CT,11,False
4,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,2032,7.65611984223897e+16,CT,76561198338258639,T,100,False


In [18]:
# TODO: Make sure this accounts for kills that don't deal damage (suicide/disconnect/fall off on vertigo)

def count_alive_and_damage(round_df):
    round_df.sort_values('RoundTick', inplace=True)

    # Tally kills to get number of players alive
    round_df['TKilled'] = (round_df.IsKill) & (round_df.VictimSide == 'T')
    round_df['CTKilled'] = (round_df.IsKill) & (round_df.VictimSide == 'CT')
    round_df['TAlive'] = 5 - round_df.TKilled.cumsum()
    round_df['CTAlive'] = 5 - round_df.CTKilled.cumsum()
    
    # Tally damage to get total team HP
    round_df['TDamaged'] = (round_df.VictimSide == 'T') * round_df.HpDamageTaken
    round_df['CTDamaged'] = (round_df.VictimSide == 'CT') * round_df.HpDamageTaken
    round_df['TTotalHp'] = 500 - round_df.TDamaged.cumsum()
    round_df['CTTotalHp'] = 500 - round_df.CTDamaged.cumsum()

    round_df.drop(['IsKill', 'TKilled', 'CTKilled', 'TDamaged', 'CTDamaged', 'HpDamageTaken'], axis=1, inplace=True)
    return round_df

damages_kills_state_df = damages_df.groupby(['MatchId', 'MapName', 'RoundNum']).apply(count_alive_and_damage)
damages_kills_state_df.set_index(['MatchId', 'MapName', 'RoundNum', 'RoundTick'], inplace=True)

In [19]:
# Make sure data looks good
damages_kills_state_df.describe()

,TAlive,CTAlive,TTotalHp,CTTotalHp
count,843967.0,843967.0,843967.000000,843967.000000
mean,5.0,5.0,294.680531,327.314256
std,0.0,0.0,143.010039,136.936846
min,5.0,5.0,-110.000000,-97.000000
25%,5.0,5.0,186.000000,225.000000
50%,5.0,5.0,304.000000,352.000000
75%,5.0,5.0,408.000000,446.000000
max,5.0,5.0,500.000000,500.000000


In [20]:
'DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_vertigo', 'RoundNum'

('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3',
 'de_vertigo',
 'RoundNum')

## Player Positions

In [21]:
player_frames_df = pd.read_csv('scrape_parse/PlayerFrames.csv', index_col=0, usecols=['MatchId', 'MapName', 'PlayerSteamId', 'RoundNum', 'Tick', 'Side', 'HasDefuse', 'DistToBombsiteA', 'DistToBombsiteB'])
player_frames_df = keep_in_round_time(player_frames_df)
player_frames_df.head()


,MatchId,MapName,RoundNum,RoundTick,Side,PlayerSteamId,HasDefuse,DistToBombsiteA,DistToBombsiteB
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,252,CT,76561198135228658,False,28,36
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,252,CT,76561197963277583,True,13,39
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,252,CT,76561198061789000,False,9,59
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,252,CT,76561198286138224,False,14,40
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,252,CT,76561198051070311,False,13,41


In [22]:
player_frames_df.drop('PlayerSteamId', axis=1, inplace=True)
t_player_frames_df = player_frames_df[player_frames_df.Side == 'T'].drop(['Side', 'HasDefuse'], axis=1)  # Only Counter-Terrorists can buy defuse kits
ct_player_frames_df = player_frames_df[player_frames_df.Side == 'CT'].drop('Side', axis=1)

t_frames_df = t_player_frames_df.groupby(['MatchId', 'MapName', 'RoundNum', 'RoundTick']).aggregate({'DistToBombsiteA': 'min', 'DistToBombsiteB': 'min'})
ct_frames_df = ct_player_frames_df.groupby(['MatchId', 'MapName', 'RoundNum', 'RoundTick']).aggregate({'DistToBombsiteA': 'min', 'DistToBombsiteB': 'min'})
t_frames_df.columns = ['ClosestDistToA', 'ClosestDistToB']
ct_frames_df.columns = ['ClosestDistToA', 'ClosestDistToB']

def lag_distance(df):
    df.sort_values('RoundTick', inplace=True)
    df['ChangeClosestDistToA'] = df['ClosestDistToA'] - df['ClosestDistToA'].shift(1)
    df['ChangeClosestDistToB'] = df['ClosestDistToB'] - df['ClosestDistToB'].shift(1)
    df['ChangeClosestDistToA'][0] = 0
    df['ChangeClosestDistToB'][0] = 0
    return df

t_frames_df = t_frames_df.groupby(['MatchId', 'MapName', 'RoundNum']).apply(lag_distance)
ct_frames_df = ct_frames_df.groupby(['MatchId', 'MapName', 'RoundNum']).apply(lag_distance)

position_state_df = pd.merge(t_frames_df, ct_frames_df, 'outer', ['MatchId', 'MapName', 'RoundNum', 'RoundTick'], suffixes=('_T', '_CT'), validate='one_to_one')


In [23]:
position_state_df.head(20)

ClosestDistToA_T  \
MatchId                                            MapName  RoundNum RoundTick                     
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1        104                    32.0   
                                                                     362                    33.0   
                                                                     620                    33.0   
                                                                     878                    33.0   
                                                                     1136                   33.0   
                                                                     1394                   33.0   
                                                                     1652                   36.0   
                                                                     1912                   37.0   
                                                                     2170                   34.0   
                                                                     2428                   34.0   
                                                                     2686                   33.0   
                                                                     2944                   32.0   
                                                                     3202                   32.0   
                                                                     3460                   34.0   
                                                                     3718                   34.0   
                                                                     3976                   34.0   
                                                                     4234                   34.0   
                                                                     4492                   34.0   
                                                                     4750                   34.0   
                                                                     5008                   34.0   

                                                                                ClosestDistToB_T  \
MatchId                                            MapName  RoundNum RoundTick                     
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1        104                    30.0   
                                                                     362                    26.0   
                                                                     620                    19.0   
                                                                     878                    17.0   
                                                                     1136                   16.0   
                                                                     1394                   13.0   
                                                                     1652                   10.0   
                                                                     1912                    6.0   
                                                                     2170                    3.0   
                                                                     2428                    3.0   
                                                                     2686                    2.0   
                                                                     2944                    3.0   
                                                                     3202                    3.0   
                                                                     3460                    3.0   
                                                                     3718                    3.0   
                                                                     3976                    3.0   
                                                                     4234      

In [24]:
position_state_df.describe()

,ClosestDistToA_T,ClosestDistToB_T,ChangeClosestDistToA_T,ChangeClosestDistToB_T,ClosestDistToA_CT,ClosestDistToB_CT,ChangeClosestDistToA_CT,ChangeClosestDistToB_CT
count,1.870602e+06,1.870602e+06,1.870602e+06,1.870602e+06,1.870349e+06,1.870349e+06,1.870349e+06,1.870349e+06
mean,1.862659e+01,1.998202e+01,-4.461505e-01,-4.155769e-01,9.562168e+00,1.063956e+01,-1.054402e-01,-1.349507e-01
std,1.292742e+01,1.130454e+01,2.430936e+00,3.448913e+00,7.848227e+00,8.576449e+00,2.656780e+00,3.104147e+00
min,0.000000e+00,0.000000e+00,-4.200000e+01,-5.200000e+01,0.000000e+00,0.000000e+00,-4.700000e+01,-4.300000e+01
25%,8.000000e+00,1.100000e+01,-1.000000e+00,-1.000000e+00,5.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
50%,1.700000e+01,2.000000e+01,0.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00
75%,2.500000e+01,2.700000e+01,0.000000e+00,0.000000e+00,1.200000e+01,1.500000e+01,0.000000e+00,0.000000e+00
max,6.900000e+01,6.700000e+01,4.000000e+01,5.400000e+01,6.600000e+01,6.800000e+01,4.800000e+01,4.400000e+01


## Putting it all together

In [25]:
bomb_state_df['EventType'] = 'BombPlant'
damages_kills_state_df['EventType'] = 'DamageDealt'
position_state_df['EventType'] = 'PositionUpdate'

In [26]:
game_states_df = pd.concat([bomb_state_df, damages_kills_state_df, position_state_df], axis=0)
game_states_df = game_states_df.sort_index()
game_states_df.reset_index(inplace=True)

In [27]:
game_states_df.columns

Index(['MatchId', 'MapName', 'RoundNum', 'RoundTick', 'BombPlantedA',
       'BombPlantedB', 'AttackerSide', 'VictimSide', 'AttackerSteamId',
       'EventType', 'VictimSteamId', 'TAlive', 'CTAlive', 'TTotalHp',
       'CTTotalHp', 'ClosestDistToA_T', 'ClosestDistToB_T',
       'ChangeClosestDistToA_T', 'ChangeClosestDistToB_T', 'ClosestDistToA_CT',
       'ClosestDistToB_CT', 'ChangeClosestDistToA_CT',
       'ChangeClosestDistToB_CT'],
      dtype='object')

In [28]:
def fill_states(df):
    fill_cols = ['BombPlantedA', 'BombPlantedB', 'TAlive', 'CTAlive', 'TTotalHp', 'CTTotalHp', 'ClosestDistToA_T', 'ClosestDistToB_T', 'ChangeClosestDistToA_T', 'ChangeClosestDistToB_T', 'ClosestDistToA_CT', 'ClosestDistToB_CT', 'ChangeClosestDistToA_CT', 'ChangeClosestDistToB_CT']
    df[fill_cols] = df[fill_cols].fillna(method='ffill')
    dist_names = ['ClosestDistToA_T', 'ClosestDistToB_T', 'ClosestDistToA_CT', 'ClosestDistToB_CT']
    df[dist_names] = df[dist_names].fillna(method='bfill')
    change_names = ['ChangeClosestDistToA_T', 'ChangeClosestDistToB_T', 'ChangeClosestDistToA_CT', 'ChangeClosestDistToB_CT']
    df[change_names] = df[change_names].fillna(0)
    df['BombPlantedA'] = df['BombPlantedA'].fillna(False)
    df['BombPlantedB'] = df['BombPlantedB'].fillna(False)
    df['TAlive'] = df['TAlive'].fillna(5)
    df['CTAlive'] = df['CTAlive'].fillna(5)
    df['TTotalHp'] = df['TTotalHp'].fillna(5)
    df['CTTotalHp'] = df['CTTotalHp'].fillna(5)
    return df

game_states_df = game_states_df.groupby(['MatchId', 'MapName', 'RoundNum']).apply(fill_states)
game_states_df.head()

,MatchId,MapName,RoundNum,RoundTick,BombPlantedA,BombPlantedB,AttackerSide,VictimSide,AttackerSteamId,EventType,...,TTotalHp,CTTotalHp,ClosestDistToA_T,ClosestDistToB_T,ChangeClosestDistToA_T,ChangeClosestDistToB_T,ClosestDistToA_CT,ClosestDistToB_CT,ChangeClosestDistToA_CT,ChangeClosestDistToB_CT
0,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,104,False,False,NaN,NaN,NaN,PositionUpdate,...,5.0,5.0,32.0,30.0,0.0,0.0,6.0,17.0,0.0,0.0
1,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,362,False,False,NaN,NaN,NaN,PositionUpdate,...,5.0,5.0,33.0,26.0,1.0,-4.0,15.0,14.0,9.0,-3.0
2,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,620,False,False,NaN,NaN,NaN,PositionUpdate,...,5.0,5.0,33.0,19.0,0.0,-7.0,7.0,13.0,-8.0,-1.0
3,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,646,False,False,T,CT,7.656119811125186e+16,DamageDealt,...,500.0,415.0,33.0,19.0,0.0,-7.0,7.0,13.0,-8.0,-1.0
4,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,878,False,False,NaN,NaN,NaN,PositionUpdate,...,500.0,415.0,33.0,17.0,0.0,-2.0,9.0,11.0,2.0,-2.0


In [29]:
game_states_df = game_states_df.merge(rounds_df[['MatchId', 'MapName', 'RoundNum', 'RoundEndReason', 'CTBuyType', 'TBuyType', 'WinningSide']], how='left', on=['MatchId', 'MapName', 'RoundNum'], validate='many_to_one')

In [30]:
game_states_df.head()

,MatchId,MapName,RoundNum,RoundTick,BombPlantedA,BombPlantedB,AttackerSide,VictimSide,AttackerSteamId,EventType,...,ChangeClosestDistToA_T,ChangeClosestDistToB_T,ClosestDistToA_CT,ClosestDistToB_CT,ChangeClosestDistToA_CT,ChangeClosestDistToB_CT,RoundEndReason,CTBuyType,TBuyType,WinningSide
0,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,104,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,0.0,6.0,17.0,0.0,0.0,TargetBombed,Pistol,Pistol,T
1,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,362,False,False,NaN,NaN,NaN,PositionUpdate,...,1.0,-4.0,15.0,14.0,9.0,-3.0,TargetBombed,Pistol,Pistol,T
2,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,620,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,-7.0,7.0,13.0,-8.0,-1.0,TargetBombed,Pistol,Pistol,T
3,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,646,False,False,T,CT,7.656119811125186e+16,DamageDealt,...,0.0,-7.0,7.0,13.0,-8.0,-1.0,TargetBombed,Pistol,Pistol,T
4,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,878,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,-2.0,9.0,11.0,2.0,-2.0,TargetBombed,Pistol,Pistol,T


In [31]:
game_states_df.shape[0]

2736698

In [32]:
bad_rounds = game_states_df[(game_states_df.TTotalHp < 0) | (game_states_df.CTTotalHp < 0)][['MatchId', 'MapName', 'RoundNum']].drop_duplicates()
bad_rounds = bad_rounds.reindex(columns=['MatchId', 'MapName', 'RoundNum'])
#  | ((game_states_df.TTotalHp > 0) & (game_states_df.TAlive == 0)) | ((game_states_df.CTTotalHp > 0) & (game_states_df.CTAlive == 0))
# Going to leave this^^ out because it elimintes too mucn of the data (~8%) (there are a lot of rounds where an entire team is dead and they still have >0 HP, probably because a damage event was missed somewhere)
bad_game_states = game_states_df.merge(bad_rounds, how='left', validate='many_to_one', indicator=True)
game_states_df = bad_game_states[bad_game_states._merge != 'both'].drop('_merge', axis=1)
game_states_df.shape[0]

2711775

In [33]:
game_states_df.to_csv('game_states.csv', encoding='utf-8', index=False)